In [6]:
import sfmta
import os

os.environ['SHUTTLE_DB_USER'] = "postgres"
os.environ['SHUTTLE_DB_PASSWORD'] =''
os.environ['SHUTTLE_DB_HOST'] = "172.17.0.3"

In [7]:
conn = sfmta.db_connect()

Connection Created


In [8]:
sfmta.main(conn)

VBox(children=(HBox(children=(Dropdown(description='Company', index=5, options=('Storer Transportation Service DBA Storer Coachways DBA Storer San Francisco', 'Lux Leasing LLC', 'Kaiser Permanente', 'Hallcon Corporation   (Loop Transportation, Inc.)', 'Pure Luxury Transportation', 'WeDriveU', 'Royal Coach Tours', 'Compass Transportation', 'TRANSMETRO', 'Corinthian International Parking Services Inc. ', 'Bishop Ranch Transportation', 'Black Tie Transportaion', 'San Francisco Minibus'), value='WeDriveU'), Button(description='Draw/Update Map', style=ButtonStyle()), Button(description='Show Restrictions', style=ButtonStyle()), Button(description='Hide Restrictions', style=ButtonStyle()))), HBox(children=(Dropdown(description='Plate', options=('CP96872', 'CP96879', '76734M1', '14053T1', '8T47570', '8Z97897', '8Z98015', '8Z98150', '60595U1', '48361F1', 'CP83137', 'PLBUS60', 'CP96880', 'CP80923', '96562J1', '8T69481', '8Z98048', 'CP96868', '63488B2', '63002E1', '23485S1', '02418T1', '63689A1', '93057W1', '23486S1', '8Z56334', '96263J1', '59956U1', '08236Y1', 'PLBUS73', '8T47569', 'PLBUS84', 'PLBUS57', '59955U1', '8T69480', 'CP96884', '93120W1', '23965S1', 'CP77577', '', '31591E2', '58595C2', '8V53967', '62684X1', 'PLBUS76', 'CP96875', '96608J1', '8Z56587', 'EP00723', '44023L1', '93125W1', '8S75248', '64322B2', '8S63363', '15998C1', 'CP80515', '31588E2', '415640Z', '56055G1', '56054G1', '56056G1', '8V14304', '64943B2', '1654401', '23503U1', '87177R1', '8W02553', '15877V1', '16502C1', '98273k1', '93121W1', '84763A2', '44315L1', '63690A1', '63486B2', '63487B2', '96573J1', '8Z98043', '23502U1', '34763H2', '88358R1', '71947Z1', '04167T1', '08238Y1', '17447U1', '13861L1', '8Z98026', '11784N1', '03670T1', 'EP00725', '46984N1', '62686X1', '79508Y1', '64319B2', '08235Y1', '63301E1', '15336M1', '58368C2', '98084K1', '15925V1', '14237D2', '79505Y1', '03668T1', '04164T1', '84762A2', '02416T1', '83965D1', '88357R1', '08355Y1', '07757H1', '98387K1', '71946Z1', 'PLBUS56', '61862J1', '04165T1', '63687A1', '96403J1', 'BUJ6137', '98275k1', '8S75246', '69570X1', '8S63365', '96122J1', '58367C2', 'BUE3848', '78459N1', '98122S1', '69220X1', '63489B2', '51761B1', '34985H2', '8Y70385', '64320B2', '85844A2', '88360R1', '62679X1', '64097B2', '85651G1', '53600P1', 'EP00710', '66225X1', '98272k1', '65270V1', '48042S1', '96618J1', '80696N1', '8Z90217', '63142E1', '34764H2', '80693N1', '8Z56585', '61740B2', '96405J1', '71921Z1', '64079B2', '85513G1', '96480J1', '79506Y1', 'EP00711', '81199Y1', '96219J1', '64942B2', '58307C2', '66703X1', '85628G1', '58369C2', '96574J1', '64383B2', '60497U1', '62678X1', '33771A2', '08719Y1', '66230X1', '30689E2', '65274V1', '09469D2', '85785G1', '65275V1', '87178R1', '35368K1', '96402J1', '92396W1', '92571W1', '66957V1', '98778E2', '66484V1', 'EP00724', '80694N1', '63688A1', '87181R1', '8Z98046', '34987H2', '96406J1', '71922Z1', '35021H2', '92363W1', '8Y70386', '16543E1', '92361W1', '48362F1', '16280V1', 'CP80825', '09803D2', '33195A2', '08354Y1', '8Z97907', 'EP00709', '33193A2', '14326P1', '8Y70294', '08352Y1', 'CP96874', '23501U1', '30692E2', 'EP00726', '03603T1', '52035B1', '48382F1', '92364W1', '85787G1', '62683X1', '03669T1', '87180R1', '59410T1', '09500D2', '85784G1', '08237Y1', '16503C1', '09502D2', '14302D2', '98083K1', '68932X1', '33194A2', '88355R1', 'EP00708', '62677X1', '23207U1', '63295E1', '02455T1', '13859L1', '69569X1', '31975T1', '15876V1', '63584E1', '78461N1', '69568X1', '15878V1', '16500C1', '07851H1', '66222V1', '62681X1', '23504U1', '59499C2', '8Z56577', '92362W1', '62675X1', '19021U1', '34986H2', '48043S1', '93126W1', '58366C2', '66221V1', '62685X1', '59411T1', '61715J1', '62687X1', '30691E2', '79507Y1', '63719E1', '83863A2', '59959U1', '63486A1', '34988H2', '59498C2', '63908B2', '85443G1', 'PLBUS77', 'CP80824', '15337M1', '66223X1', '92452W1', '61499B2', '8Z97821', '8Z97908', '48381F1', '13863L1', '76421Y1', '96265J1', '09715D2', '59608U1', '96302J1', '95105V1', '96616J1', 'EP00707', '76512Y1', '30688E2', '9

Map(basemap={'url': 'http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png', 'max_zoom': 18, 'attribution': 'Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'Hydda.Full'}, center=[37.79481, -122.41186], default_tiles=TileLayer(attribution='Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', min_zoom=1, name='Hydda.Full', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'), layers=(TileLayer(attribution='Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', min_zoom=1, name='Hydda.Full', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'), Polyline(color='black', fill_color='black', options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'no_clip', 'opacity', 'pointer_events', 'smooth_factor', 'stroke', 'weight'])), layout=Layout(align_self='stretch', height='650px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'])

---

# WIP--------

## summary table
- Summary table by CNN 
- Summary table by stops (every second within 50 feet of a stop point, it will be a stop)
- periodic calculations (populate the summary tables, cron jobs?, another server - lambda), how often will they be peformed
- cron job to run in postgres dd?
- python function to start


## SQL queries
- volume per CNN
- route(trip) identification
- sopping and update stoping table
- point inside/near CNN polygon
- distance from polygon
-

## dropdown UI
- (done) timestamp with date fields
- how to update parameters with each field


## Implementation
1. upsert? function to tie each individual point to cnn, use postgres windowing fucntion. gropupby and min on distance to cnn (to associate cnn to points)
2. stops, companies and vehicles added to the db schema
3. summary of stops and frequency of stops, add table of stops
4. self report stops by changing a field, initially look at self reporting and see how accurate it is
- query to look at min/max time stamp with every group of points to classify stops
- flter for specific stop



### When should we persist data?
- summaries by CNN (every half hour at a 15min lag)
- 

- We should compare apples to apples both solutions